In [20]:
%matplotlib notebook

import matplotlib.pylab as plt
import numpy as np
import seaborn as sns; sns.set()

import keras
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam
import salty
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from random import shuffle

In [12]:
devmodel = salty.aggregate_data(['density'])
devmodel.Data['smiles_string'] = devmodel.Data['smiles-cation'] + "." + devmodel.Data['smiles-anion']
values = devmodel.Data['smiles_string']
print(values.shape)
values2 = values.drop_duplicates();
smile_max_length = values2.map(len).max()

C:\Users\james\Anaconda3\lib\site-packages\pandas\core\generic.py:4388: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
C:\Users\james\Anaconda3\lib\site-packages\pandas\core\generic.py:4389: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)


(28952,)


In [13]:
def pad_smiles(smiles_string):
     if len(smiles_string) < smile_max_length:
            return smiles_string + " " * (smile_max_length - len(smiles_string))

In [14]:
padded_smiles =  [pad_smiles(i) for i in values2 if pad_smiles(i)]

In [21]:
shuffle(padded_smiles)

In [42]:
def create_char_list(char_set, smile_series):
    for smile in smile_series:
        char_set.update(set(smile))
    return char_set

In [43]:
char_set = set()
char_set = create_char_list(char_set, padded_smiles)

In [44]:
char_set

{' ',
 '#',
 '(',
 ')',
 '+',
 '-',
 '.',
 '1',
 '2',
 '3',
 '=',
 '@',
 'B',
 'C',
 'F',
 'H',
 'I',
 'N',
 'O',
 'P',
 'S',
 '[',
 '\\',
 ']',
 'c',
 'e',
 'l',
 'n',
 'r'}

In [62]:
char_list = list(char_set)
chars_in_dict = len(char_list)
char_to_index = dict((c, i) for i, c in enumerate(char_list))
index_to_char = dict((i, c) for i, c in enumerate(char_list))

In [63]:
char_to_index

{'\\': 0,
 'F': 1,
 'n': 2,
 '2': 3,
 'c': 4,
 'B': 5,
 'e': 6,
 '#': 7,
 '@': 8,
 '-': 9,
 '3': 10,
 '[': 11,
 'l': 12,
 ')': 13,
 'O': 14,
 '=': 15,
 'H': 16,
 ']': 17,
 'P': 18,
 '.': 19,
 'C': 20,
 'r': 21,
 'I': 22,
 'N': 23,
 '1': 24,
 '(': 25,
 'S': 26,
 ' ': 27,
 '+': 28}

In [68]:
X_train = np.zeros((len(padded_smiles), smile_max_length, chars_in_dict), dtype=np.float32)

In [69]:
X_train.shape

(502, 105, 29)

In [71]:
for i, smile in enumerate(padded_smiles):
    for j, char in enumerate(smile):
        X_train[i, j, char_to_index[char]] = 1

In [72]:
X_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1.

In [73]:
X_train, X_test = train_test_split(X_train, test_size=0.33, random_state=42)

In [74]:
X_train.shape

(336, 105, 29)

In [84]:
# need to build RNN to encode. some issues include what the 'embedded dimension' is (vector length of embedded sequence)